In [5]:
import os
import csv
import random

import torch as th
import torchaudio as tha
from tqdm import tqdm
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

from TTS.tts.configs.shared_configs import BaseDatasetConfig
from TTS.tts.datasets import load_tts_samples
from TTS.tts.configs.glow_tts_config import GlowTTSConfig
from TTS.tts.models.glow_tts import GlowTTS
from TTS.tts.utils.text.tokenizer import TTSTokenizer
from TTS.utils.audio import AudioProcessor

from trainer import Trainer, TrainerArgs

In [2]:
def load_df_from_tsv(path: str):
    return pd.read_csv(
        path,
        sep="\t",
        header=0,
        encoding="utf-8",
        escapechar="\\",
        quoting=csv.QUOTE_NONE,
        na_filter=False,
    )

In [3]:
device = 'cuda'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

# Generate German texts translated from English references in train_st_sv-SE_en

In [4]:
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-de")
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-de").to(device)

In [ ]:
input = ["Hello world.", "This is good!"]
tokenized_input = tokenizer(input, padding=True, return_tensors="pt").to(device)
output = model.generate(**tokenized_input)
translation = tokenizer.batch_decode(output, skip_special_tokens=True)

In [ ]:
translation

['Hallo Welt.', 'Das ist gut!']

In [5]:
path = '/mnt/raid0/siqi/datasets/covost2/sv-SE/train_st_sv-SE_en.tsv'
sv_df = load_df_from_tsv(path)
en_ref = sv_df['tgt_text'].tolist()

In [6]:
batch_size = 500
de_trans = []
for idx in tqdm(range(0, len(en_ref), batch_size)):
    input = en_ref[idx : idx + batch_size]
    tokenized_input = tokenizer(input, padding=True, return_tensors="pt").to(device)
    output = model.generate(**tokenized_input)
    translation = tokenizer.batch_decode(output, skip_special_tokens=True)
    de_trans.extend(translation)

100%|██████████| 5/5 [00:12<00:00,  2.53s/it]


# Train TTS model on German texts

## Build TTS Dataset

In [12]:
path = '/mnt/raid0/siqi/datasets/covost2/de/train_st_de_en.tsv'
df = load_df_from_tsv(path)
de_transcript = df['src_text'].tolist()
de_audio = df['audio'].tolist()

In [13]:
metadata_path = '/mnt/raid0/siqi/datasets/covost2/de/train_tts_de.txt'
with open(metadata_path, 'w') as w:
    for transcript, audio in zip(de_transcript, de_audio):
        w.write('{}|{}\n'.format(audio, transcript))

In [14]:
de_root = '/mnt/raid0/siqi/datasets/covost2/de'
dataset_config = BaseDatasetConfig(
    name="ljspeech", meta_file_train='train_tts_de.txt', language="de", path=de_root
)

In [15]:
def formatter(root_path, manifest_file, **kwargs):  # pylint: disable=unused-argument
    """Assumes each line as ```<filename>|<transcription>```
    """
    txt_file = os.path.join(root_path, manifest_file)
    items = []
    speaker_name = "my_speaker"
    with open(txt_file, "r", encoding="utf-8") as ttf:
        for line in ttf:
            cols = line.split("|")
            wav_file = os.path.join(root_path, "16kHz", cols[0])
            text = cols[1]
            items.append({"text":text, "audio_file":wav_file, "speaker_name":speaker_name})
    return items

In [16]:
train_samples, eval_samples = load_tts_samples(dataset_config, eval_split=True, formatter=formatter)

 | > Found 127824 files in /mnt/raid0/siqi/datasets/covost2/de


In [17]:
len(train_samples), len(eval_samples)

(126546, 1278)

## Train a TTS Model

In [19]:
save_dir = '/mnt/raid0/siqi/checkpoints/de_tts'
config = GlowTTSConfig(
    batch_size=32,
    eval_batch_size=16,
    num_loader_workers=4,
    num_eval_loader_workers=4,
    run_eval=True,
    test_delay_epochs=-1,
    epochs=1000,
    text_cleaner="phoneme_cleaners",
    use_phonemes=True,
    phoneme_language="de-de",
    phoneme_cache_path=os.path.join(save_dir, "phoneme_cache"),
    print_step=25,
    print_eval=False,
    mixed_precision=True,
    output_path=save_dir,
    datasets=[dataset_config],
)

In [21]:
ap = AudioProcessor.init_from_config(config)
tokenizer, config = TTSTokenizer.init_from_config(config)

 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:True
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:0.0
 | > pitch_fmax:640.0
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:45
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024


In [22]:
model = GlowTTS(config, ap, tokenizer, speaker_manager=None)

In [24]:
trainer = Trainer(
    TrainerArgs(), config, save_dir, model=model, train_samples=train_samples, eval_samples=eval_samples
)

RuntimeError:  [!] 8 active GPUs. Define the target GPU by `CUDA_VISIBLE_DEVICES`. For multi-gpu training use `TTS/bin/distribute.py`.

## Inference with TTS Model

In [9]:
# coqai
dest_dir = '/mnt/raid0/siqi/datasets/covost2/sv-SE/16kHz_de'
os.makedirs(dest_dir, exist_ok=True)
sv_audio = sv_df['audio'].tolist()
for de, audio in tqdm(zip(de_trans, sv_audio), total=len(de_trans)):
    os.system('tts --text "{}" --model_name tts_models/de/thorsten/tacotron2-DCA --use_cuda 1 --out_path {}'.format(
        de, os.path.join(dest_dir, audio)
    ))

  0%|          | 0/2160 [00:00<?, ?it/s]

 > tts_models/de/thorsten/tacotron2-DCA is already downloaded.
 > vocoder_models/de/thorsten/fullband-melgan is already downloaded.
 > Using model: Tacotron2
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:True
 | > symmetric_norm:True
 | > mel_fmin:50.0
 | > mel_fmax:8000.0
 | > pitch_fmin:0.0
 | > pitch_fmax:640.0
 | > spec_gain:1.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:50
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:/home/siqi/.local/share/tts/tts_models--de--thorsten--tacotron2-DCA/scale_stats.npy
 | > base:10
 | > hop_length:256
 | > win_length:1024
 > Model's reduct

  0%|          | 1/2160 [00:06<3:53:38,  6.49s/it]

 > tts_models/de/thorsten/tacotron2-DCA is already downloaded.
 > vocoder_models/de/thorsten/fullband-melgan is already downloaded.
 > Using model: Tacotron2
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:True
 | > symmetric_norm:True
 | > mel_fmin:50.0
 | > mel_fmax:8000.0
 | > pitch_fmin:0.0
 | > pitch_fmax:640.0
 | > spec_gain:1.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:50
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:/home/siqi/.local/share/tts/tts_models--de--thorsten--tacotron2-DCA/scale_stats.npy
 | > base:10
 | > hop_length:256
 | > win_length:1024
 > Model's reduct

  0%|          | 2/2160 [00:13<4:04:06,  6.79s/it]

...

 > tts_models/de/thorsten/tacotron2-DCA is already downloaded.
 > vocoder_models/de/thorsten/fullband-melgan is already downloaded.
 > Using model: Tacotron2
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:True
 | > symmetric_norm:True
 | > mel_fmin:50.0
 | > mel_fmax:8000.0
 | > pitch_fmin:0.0
 | > pitch_fmax:640.0
 | > spec_gain:1.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:50
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:/home/siqi/.local/share/tts/tts_models--de--thorsten--tacotron2-DCA/scale_stats.npy
 | > base:10
 | > hop_length:256
 | > win_length:1024
 > Model's reduct

 11%|█         | 233/2160 [27:31<4:27:15,  8.32s/it]

 > tts_models/de/thorsten/tacotron2-DCA is already downloaded.
 > vocoder_models/de/thorsten/fullband-melgan is already downloaded.
 > Using model: Tacotron2
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:True
 | > symmetric_norm:True
 | > mel_fmin:50.0
 | > mel_fmax:8000.0
 | > pitch_fmin:0.0
 | > pitch_fmax:640.0
 | > spec_gain:1.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:50
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:/home/siqi/.local/share/tts/tts_models--de--thorsten--tacotron2-DCA/scale_stats.npy
 | > base:10
 | > hop_length:256
 | > win_length:1024
 > Model's reduct

 11%|█         | 234/2160 [27:38<4:14:23,  7.93s/it]

 > tts_models/de/thorsten/tacotron2-DCA is already downloaded.
 > vocoder_models/de/thorsten/fullband-melgan is already downloaded.
 > Using model: Tacotron2
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:True
 | > symmetric_norm:True
 | > mel_fmin:50.0
 | > mel_fmax:8000.0
 | > pitch_fmin:0.0
 | > pitch_fmax:640.0
 | > spec_gain:1.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:50
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:/home/siqi/.local/share/tts/tts_models--de--thorsten--tacotron2-DCA/scale_stats.npy
 | > base:10
 | > hop_length:256
 | > win_length:1024
 > Model's reduct

In [7]:
# silero
language = 'de'
speaker = 'thorsten_v2'
sample_rate = 16000
device = th.device('cuda')
model, example_text = th.hub.load(repo_or_dir='snakers4/silero-models',
                                    model='silero_tts',
                                    language=language,
                                    speaker=speaker)
model.to(device)  # gpu or cpu

Using cache found in /home/siqi/.cache/torch/hub/snakers4_silero-models_master


In [8]:
audios = []
batch_size = 500
for idx in tqdm(range(0, len(de_trans), batch_size)):
    batch_audio = model.apply_tts(texts=de_trans[idx : idx + batch_size], sample_rate=sample_rate)
    audios.extend(batch_audio)

  0%|          | 0/5 [00:00<?, ?it/s]/mnt/nvme/siqi/anaconda3/envs/tts/lib/python3.8/site-packages/torch/nn/modules/module.py:1110: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at  /opt/conda/conda-bld/pytorch_1646755903507/work/aten/src/ATen/native/cudnn/RNN.cpp:926.)
  return forward_call(*input, **kwargs)
100%|██████████| 5/5 [00:20<00:00,  4.13s/it]


In [11]:
dirname = '/mnt/raid0/siqi/datasets/covost2/sv-SE/16kHz_de_silero'
sv_audio = sv_df['audio'].tolist()
for audio, idx in zip(audios, sv_audio):
    tha.save(os.path.join(dirname, idx), audio.unsqueeze(0), sample_rate=sample_rate)

## VAD to remove silence

In [11]:
model, utils = th.hub.load(repo_or_dir='snakers4/silero-vad',
                            model='silero_vad',
                            force_reload=True,
                            onnx=False)

(get_speech_timestamps,
 save_audio,
 read_audio,
 VADIterator,
 collect_chunks) = utils

model = model.to(device)

Downloading: "https://github.com/snakers4/silero-vad/archive/master.zip" to /home/siqi/.cache/torch/hub/master.zip


In [14]:
sampling_rate = 16000
dirname = '/mnt/raid0/siqi/datasets/covost2/sv-SE/16kHz_de_silero'
new_dirname = '/mnt/raid0/siqi/datasets/covost2/sv-SE/16kHz_de_silero_vad'
os.makedirs(new_dirname, exist_ok=True)
for audio in tqdm(os.listdir(dirname)):
    path = os.path.join(dirname, audio)
    new_path = os.path.join(new_dirname, audio)
    wav = read_audio(path, sampling_rate=sampling_rate).to(device)
    speech_timestamps = get_speech_timestamps(wav, model, sampling_rate=sampling_rate)
    save_audio(new_path, collect_chunks(speech_timestamps, wav.cpu()), sampling_rate=sampling_rate)

  0%|          | 0/2160 [00:00<?, ?it/s]/mnt/nvme/siqi/anaconda3/envs/tts/lib/python3.8/site-packages/torch/nn/modules/module.py:1110: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at  /opt/conda/conda-bld/pytorch_1646755903507/work/aten/src/ATen/native/cudnn/RNN.cpp:926.)
  return forward_call(*input, **kwargs)
100%|██████████| 2160/2160 [09:07<00:00,  3.95it/s]


In [6]:
dirname = '/mnt/raid0/siqi/datasets/covost2/sv-SE/16kHz_de_silero'
new_dirname = '/mnt/raid0/siqi/datasets/covost2/sv-SE/16kHz_de_silero_vad'
audios = os.listdir(dirname)
random.shuffle(audios)
for audio in tqdm(audios[:100]):
    os.system('cp {} ./before_vad'.format(os.path.join(dirname, audio)))
    os.system('cp {} ./after_vad'.format(os.path.join(new_dirname, audio)))

100%|██████████| 100/100 [00:04<00:00, 22.46it/s]
